# **eWOM and NFTs - Data Scrapping and Sentiment Analysis**


---


# **1. Introduction:**

A sentiment analysis is performed in a series of tweets related to different
NFT assets, linking OpenSea and Twitter parameters with tweet valence. An
ordered logistic regression is carried out to assess the impact of not only tweet sentiment, but different Twitter and OpenSea metrics on the value of NFTs.

##Scraping OpenSea

In [ ]:
!pip install opensea
import requests
import pandas as pd
import json
import random
import datetime
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Dataset:**

The NFT price dataset uses data scraped from the OpenSea API. A timeframe of four months was set from January 2022 to April 2022 and only assets that were actually sold were considered.

In [ ]:
asset_events = [] #creating a list?
url = "https://api.opensea.io/api/v1/events"
headers = {"Accept": "application/json",
"X-API-KEY": "xxxxxxxxxxxxxxxxxxxxxxxx"}

begindate = datetime.datetime.strptime('2022-01-01','%Y-%m-%d')
enddate = datetime.datetime.strptime('2022-04-30','%Y-%m-%d')
#it collects 50 assets from everyday between begindate and enddate
while begindate<enddate:
    print(begindate)
    #rint = random.randint(1,364)
    date1 = begindate
    date2 = begindate + datetime.timedelta(days=1)
    date1 = datetime.datetime.strftime(date1,'%Y-%m-%d')
    date2 = datetime.datetime.strftime(date2,'%Y-%m-%d')
    url = 'https://api.opensea.io/api/v1/events?occurred_before=' + date2 + 'T00%3A00%3A00&occurred_after=' + date1 + 'T00%3A00%3A00'
    params = {'limit':'50','event_type':'successful'}

    response = requests.get(url, headers=headers,params=params)
    print(response)
    x = response.json()
    asset_events = asset_events + x['asset_events']
    begindate = begindate + datetime.timedelta(days=1)

In [ ]:
asset_events_df = pd.json_normalize(asset_events)

In [ ]:
asset_events_df

In [ ]:
asset_events_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/events_keys.csv',index=False)

**Data Cleaning**
For ease of further analysis the name of the assets were cleaned, eliminating spaces and punctuation symbols. The price in the dataset is given in the cryptocurrency ethereum or wei. This price comes together with the exchange rate from ETH to USD at the time of the purchase, which is used to turn all ETH prices to USD prices for better analysis and interpretation.

In [ ]:
#cleaning name column fields to erase punctuation and digits so it works better with Twitter's snscrape
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace(r'[^\w\s]+', '')
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace('\d+', '')
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace(' ', '')

In [ ]:
asset_events_df.info(verbose = True)

In [ ]:
#cleaning total price, transform from wei to eth and turn total price to float
asset_events_df['total_price'] = asset_events_df['total_price'].astype(float, errors = 'raise')
asset_events_df['total_price'] = asset_events_df['total_price']/10.**18

In [ ]:
asset_events_df['quantity'] = pd.to_numeric(asset_events_df['quantity'],errors = 'coerce')

In [ ]:
asset_events_df['payment_token.usd_price'] = pd.to_numeric(asset_events_df['payment_token.usd_price'],errors = 'coerce')

In [ ]:
#Calculating sales price in USD.
asset_events_df['total_price_usd'] = asset_events_df['total_price'] * asset_events_df['payment_token.usd_price']

In [ ]:
asset_events_df['total_price_usd']

In [ ]:
asset_events_df = asset_events_df[['total_price','quantity','collection_slug','asset.num_sales','asset.name','payment_token.symbol','payment_token.usd_price','asset.asset_contract.address', 'asset.token_id', 'dev_seller_fee_basis_points']]

In [ ]:
asset_events_df

In [ ]:
asset_events_df[asset_events_df['payment_token.symbol'].isna()]#1970 rows

In [ ]:
asset_events_df['payment_token.symbol'].isna().sum()

1970

In [ ]:
asset_events_df['payment_token.usd_price'].mean()

1750.7427269932894

In [ ]:
asset_events_df['payment_token.symbol'].unique()

array([nan, 'ETH', 'WETH', 'ASH', 'GALA', 'CUBE'], dtype=object)

In [ ]:
asset_events_eth_df = asset_events_df.loc[(asset_events_df['payment_token.symbol'] == 'ETH') | (asset_events_df['payment_token.symbol'] =='WETH')]

In [ ]:
#Calculating sales price in USD.
asset_events_eth_df['total_price_usd'] = asset_events_eth_df['total_price'] * asset_events_eth_df['payment_token.usd_price']

In [ ]:
asset_events_eth_df

,total_price,quantity,collection_slug,asset.num_sales,asset.name,payment_token.symbol,payment_token.usd_price,asset.asset_contract.address,asset.token_id,dev_seller_fee_basis_points,total_price_usd
2,0.7500,1,alienfrensnft,5.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,6852,700.0,893.715000
3,0.7500,1,alienfrensnft,5.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,6374,700.0,893.715000
4,0.7490,1,alienfrensnft,4.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,419,700.0,892.523380
5,0.7440,1,alienfrensnft,7.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,8812,700.0,886.565280
6,0.7400,1,alienfrensnft,3.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,7210,700.0,881.798800
...,...,...,...,...,...,...,...,...,...,...,...
5939,0.0800,1,mutantshibaclub,1.0,MutantShiba,ETH,1191.62,0xca95777971c7c438eecbd8bfc23249fbee99e716,6906,800.0,95.329600
5940,0.0040,1,decentrabis,1.0,DecentrabisNuggy,ETH,1191.62,0x6b7b0bba4b8c26680f267f5621025f146a4cbddf,2447,500.0,4.766480
5941,0.0532,1,regulars,5.0,Regular,ETH,1191.62,0x6d0de90cdc47047982238fcf69944555d27ecb25,2209,500.0,63.394184
5942,0.0900,1,e-shell,1.0,EShell,ETH,1191.62,0xe41bea6888f771a0c16d7188284522b76c135252,2061,750.0,107.245800


In [ ]:
asset_events_eth_df = asset_events_eth_df.groupby(['asset.name','collection_slug'], as_index=False).agg({'total_price': 'mean', 'quantity': 'first','asset.num_sales':'mean', 'total_price_usd':'first', 'payment_token.usd_price':'mean', 'total_price_usd':'mean' })

In [ ]:
#Agreggate by asset name and collection slug
asset_events_eth_df = asset_events_eth_df.groupby(['asset.name','collection_slug'], as_index = False).sum()

In [ ]:
asset_events_eth_df.keys()

Index(['total_price', 'quantity', 'collection_slug', 'asset.num_sales',
       'asset.name', 'payment_token.symbol', 'payment_token.usd_price',
       'asset.asset_contract.address', 'asset.token_id',
       'dev_seller_fee_basis_points', 'total_price_usd'],
      dtype='object')

In [ ]:
len(asset_events_eth_df)#we have 5950 events

1530

In [ ]:
len(asset_events_eth_df['asset.name'].unique())# I am doing this to check that we don't have a lot of repeated ones
#we have 2341 unique asset events

1417

In [ ]:
asset_events_eth_df

,asset.name,collection_slug,total_price,quantity,asset.num_sales,total_price_usd,payment_token.usd_price
0,,animo-official,0.159450,1,4.000000,190.003809,1191.620
1,,avius-animae,0.042000,1,1.000000,50.048040,1191.620
2,,baby-cub-tigers,0.060000,1,2.000000,71.283000,1188.050
3,,bakedape,0.250000,1,1.000000,297.905000,1191.620
4,,barn-owlz,0.060000,1,3.000000,71.497200,1191.620
...,...,...,...,...,...,...,...
1523,xmfer,xmfers,0.186610,1,2.200000,222.261108,1191.263
1524,xmfers,0xmfers,0.003267,1,1.333333,3.892625,1191.620
1525,Ïmumonen,lootrealms,1.750000,1,3.000000,2085.335000,1191.620
1526,ἀκτῑς,futuristek-hieroglyphx,0.077700,1,21.000000,92.588874,1191.620


In [ ]:
asset_events_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/asset_events.csv',index=False)

In [ ]:
asset_events_eth_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/eth_events_2.csv',index=False)

In [ ]:
asset_events_df = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/asset_events.csv')

## Twitter Scraping


**Dataset**

Using open software ''snscrape''.

In [ ]:
!pip3 install snscrape
#!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git needs to be run in python 3.8
import snscrape.modules.twitter as sntwitter
import pandas as pd
import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
assets = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/eth_events.csv')
asset_name = assets['asset.name']
asset_name

In [ ]:
assets.info(verbose = True)

In [ ]:
# Creating list to append tweet data.
tweets_list1 = []
# Using TwitterSearchScraper to scrape data and append tweets to list.
# this enumerate function will give all the tweets
tweet_dict = {}
for x in asset_name[86:285]:
  if isinstance(x,str):
   for i, tweet in enumerate(sntwitter.TwitterSearchScraper('NFT'+ '#' + x  ).get_items()):
    #i>5000 and then break, it will stop scraping at 5001
      if i>20:
          break
      tweets_list1.append([x ,tweet.date, tweet.id, tweet.content, tweet.user.username])#tweet.retweetCount, tweet.replyCount and tweet.likeCount, tweet.user.followersCount, tweet.user.favouritesCount
      #this last part is giving us the data that we are scraping, in this case date, id, content and username
  else:
    tweets_list1.append(['']*4)
tweet_df = pd.DataFrame(tweets_list1, columns=['AssetName','Datetime', 'Tweet Id', 'Text', 'Username'])

In [ ]:
asset_name[86:185]

In [ ]:
tweet_df

In [ ]:
tweet_df.info(verbose = True)

**Twitter Data Cleaning**

Tweets are cleaned from external links and the use of emojis, to improve accuracy of the sentiment analysis.

In [ ]:
!pip install emoji --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=10de819a852d468518ec9ded0df892fd662f7d27a518d0f8faae30a14c37b2f0
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import re, string
import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

In [ ]:
len(tweet_df)

85939

In [ ]:
len(tweet_df['Text'].unique())

78132

In [ ]:
tweet_to_try = tweet_df

In [ ]:
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text)

In [ ]:
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

In [ ]:
texts_new = []
for t in tweet_to_try['Text'].iloc[10000:85939]:
    texts_new.append(strip_all_entities(strip_emoji(t)))

In [ ]:
texts_new

In [ ]:
tweet_to_try2 = tweet_to_try.iloc[10000:85939]

In [ ]:
tweet_to_try2['text_clean'] = texts_new

In [ ]:
tweet_to_try2.head()

,AssetName,Datetime,Tweet Id,Text,Username,Retweets,Replies,Likes,Followers,Favourites,Negative,Neutral,Positive,text_clean
10000,BoredMummy,2021-09-10 17:03:24+00:00,1.436375e+18,🙏 Thank you ser❤\n#mummygang we made it🎉🥰 we f...,BoredMummyWU,7.0,0.0,29.0,16290.0,8983.0,0.0,0.0,0.0,thank you ser mummygang we made it we felt th...
10001,BoredMummy,2021-09-10 15:51:37+00:00,1.436357e+18,Bored Mummy Waking Up - Trading Price Analysis...,spr3adsh33t,0.0,0.0,1.0,30936.0,10542.0,0.0,0.0,0.0,bored mummy waking up trading price analysis ...
10002,BoredMummy,2021-09-10 08:08:33+00:00,1.436240e+18,My new Bored Mummy#4289 #NewProfilePic\n\n@Bor...,itskim_D,2.0,0.0,10.0,4821.0,18159.0,0.0,0.0,0.0,my new bored mummy4289 newprofilepic nft bo...
10003,BoredMummy,2021-09-09 23:47:52+00:00,1.436114e+18,Bored Mummy Waking Up - Trading Price Analysis...,spr3adsh33t,0.0,0.0,1.0,30936.0,10542.0,0.0,0.0,0.0,bored mummy waking up trading price analysis ...
10004,BoredMummy,2021-09-08 19:23:36+00:00,1.435685e+18,Bored Mummy Waking Up - Trading Price Analysis...,spr3adsh33t,0.0,3.0,1.0,30936.0,10542.0,0.0,0.0,0.0,bored mummy waking up trading price analysis ...


## Sentiment Analysis

I follow a supervised, pre-trained, neural network model based on RoBERTa for unstructured text analysis and text mining methods that classifies tweet sentiment into positive, neutral and negative and analyses the intensity of the emotion giving the different emotion a score between 0 and 1.

In [ ]:
!pip3 install transformers

In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("text-classification", model="j-hartmann/sentiment-roberta-large-english-3-classes", return_all_scores=True)

In [ ]:
tweet_df = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/only_twitter_df.csv')

In [ ]:
compare = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/only_twitter_df.csv')

In [ ]:
tweet_df = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/sentiment_events2.csv')

In [ ]:
compare

In [ ]:
tweet_df

In [ ]:
len(tweet_df)#86024

85939

In [ ]:
tweet_df.iloc[7700:8000]

,AssetName,Datetime,Tweet Id,Text,Username,Retweets,Replies,Likes,Followers,Favourites,Negative,Neutral,Positive
7700,Bag,2012-11-03 17:13:01+00:00,2.647772e+17,@Carmela_Fortuna nga pala... That's why ur cal...,nft_garci,0.0,0.0,0.0,1019.0,26731.0,0.0,0.0,0.0
7701,Bag,2012-07-18 19:46:56+00:00,2.256781e+17,@tarawoodward17 Ergh nellie #regret #antibacte...,NFT_PROMOTERRR,0.0,1.0,0.0,6610.0,122.0,0.0,0.0,0.0
7702,Bag,2012-03-31 06:04:47+00:00,1.859709e+17,"Somebody once asked me ""what's in your #bag?"" ...",jsf_eth,0.0,0.0,0.0,423.0,6209.0,0.0,0.0,0.0
7703,Ballman,2022-06-02 10:56:49+00:00,1.532315e+18,Refresh to @ballmanproject dashboard with some...,KresimirDivic,5.0,0.0,11.0,707.0,8580.0,0.0,0.0,0.0
7704,Ballman,2022-06-01 20:04:44+00:00,1.532091e+18,Our wednesday Spaces are a must!! \nToday talk...,Shxeila,0.0,0.0,2.0,440.0,51983.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,Bean,2022-03-06 12:34:06+00:00,1.500450e+18,#sproutie #metaverse #nftgame #nft #bean https...,athayatonda,0.0,2.0,0.0,13099.0,7950.0,0.0,0.0,0.0
7996,Bean,2022-03-06 11:52:12+00:00,1.500439e+18,#sproutie #metaverse #nftgame #nft #bean https...,KafeelZhafran,0.0,0.0,0.0,18976.0,16764.0,0.0,0.0,0.0
7997,Bean,2022-03-06 07:14:36+00:00,1.500369e+18,#sproutie #metaverse #nftgame #nft #bean https...,kinghunter091,0.0,1.0,0.0,2343.0,50272.0,0.0,0.0,0.0
7998,Bean,2022-03-06 04:48:53+00:00,1.500333e+18,#sproutie #metaverse #nftgame #nft #bean https...,novie_talita,0.0,0.0,0.0,2091.0,11080.0,0.0,0.0,0.0


In [ ]:
tweet_df = tweet_df.drop(range(0,85))

In [ ]:
tweet_df.head()

In [ ]:
list_tweets = tweet_df['Text'].tolist()
list_tweets

In [ ]:
#RUN IT ONLY ONCE
tweet_df['Negative'] = [0]*len(tweet_df)
tweet_df['Neutral'] = [0]*len(tweet_df)
tweet_df['Positive'] = [0]*len(tweet_df) #creating list that we'll later fill up

In [ ]:
for i,row in (tweet_df.iloc[10000:15000].iterrows()):
  #asset_name = row['AssetName']
  text = row["Text"]
  #print('{i}-----{asset_name}-----'.format(i=i,asset_name=asset_name))
  #print(text)
  sentiments = classifier(text)
  tweet_df['Negative'].loc[i] = sentiments[0][0]['score']
  tweet_df['Neutral'].loc[i] = sentiments[0][1]['score']
  tweet_df['Positive'].loc[i] = sentiments[0][2]['score']

tweet_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/sentiment_events2.csv',index=False)


In [ ]:
tweet_df[9980:10000]

In [ ]:
tweet_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/sentiment_events3.csv',index=False)

In [ ]:
tweet_df.iloc[7000:8000]

In [ ]:
tweet_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/sentiment_events3.csv',index=False)

##Final Dataset - Merging Twitter and OpenSea Data

In [ ]:
tweet_df

,AssetName,Datetime,Tweet Id,Text,Username,Negative,Neutral,Positive
0,ALI,2022-05-31 19:10:38+00:00,1531714790296588289,BOOM!!!! I got me a SICK ALI @MuhammadAli #ali...,FrankKnuckles3,0.000217,0.014742,0.985041
1,ALI,2022-05-28 14:11:55+00:00,1530552452646174721,.0029 etc for this (8) 1/1s collection this we...,famousmoonking,0.000259,0.998367,0.001374
2,ALI,2022-05-26 23:03:16+00:00,1529961393222344704,24 hour pre #MemorialDaySale!!!\n #crypto shri...,famousmoonking,0.000584,0.930651,0.068765
3,ALI,2022-05-20 14:46:16+00:00,1527661994895020034,hi.\n8 of these @ .01 each for the weekend.\nI...,famousmoonking,0.000171,0.998158,0.001671
4,ALI,2022-05-10 03:46:15+00:00,1523872016410898432,Limited edition of 9 hand made 1/1 Ali post ca...,famousmoonking,0.000234,0.998247,0.001519
...,...,...,...,...,...,...,...,...
2011,CapybarasCountryClub,2022-02-27 21:56:10+00:00,1498054378342735875,My most recent #fanart for #capybarascountrycl...,hxrnyNFT,0.000148,0.997976,0.001876
2012,CapybarasCountryClub,2022-01-07 22:24:59+00:00,1479579847991205894,Do you like what you have seen so far? 👀🔥\n\n#...,CapybarasCC,0.000165,0.001351,0.998484
2013,CapybarasCountryClub,2022-01-07 11:13:52+00:00,1479410953947164674,I love to see our community growing organicall...,CapybarasCC,0.000128,0.001793,0.998079
2014,CapybarasCountryClub,2022-01-06 08:14:47+00:00,1479003498104725507,"GM !!! \n\nNew day, new banner ! \n\nOG roles ...",CapybarasCC,0.000188,0.996583,0.003230


In [ ]:
assets[86:285]

,asset.name,collection_slug,total_price,quantity,asset.num_sales,payment_token.usd_price,total_price_usd
86,AKCredit,a0k1verse-credits,1.68900,7,8800.0,7901.61,3336.289330
87,ALI,alinft-official,0.75900,4,20.0,7901.08,1499.229930
88,ALPACADABRAD,alpacadabraz-3d,0.28700,1,1.0,1975.27,566.902490
89,AMBIVERT,carmeljenkinart,0.10000,1,1.0,1975.27,197.527000
90,AOArtBall,ao-art-ball,2.64359,13,21.0,25678.51,5221.804019
...,...,...,...,...,...,...,...
280,CameraPerson,camera-person-nft,0.07800,2,2.0,3950.54,154.071060
281,CandyHunter,candy-hunters,0.39700,6,13.0,11851.62,784.182190
282,CannonPlay,snapshots-of-sydney,0.10000,1,3.0,1975.27,197.527000
283,CapybarasCountryClub,capybarascountryclub,0.21780,4,6.0,7901.08,430.213806


In [ ]:
almost_final_df = pd.merge(assets[86:285], tweet_df, left_on="asset.name", right_on="AssetName").drop('AssetName', axis = 1)

In [ ]:
almost_final_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/almost_final_df.csv',index=False)

In [ ]:
almost_final_df = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/almost_final_df.csv')
almost_final_df

,asset.name,collection_slug,total_price,quantity,asset.num_sales,payment_token.usd_price,total_price_usd,Datetime,Tweet Id,Text,Username,Negative,Neutral,Positive
0,ALI,alinft-official,0.7590,4,20.0,7901.08,1499.229930,2022-05-31 19:10:38+00:00,1531714790296588289,BOOM!!!! I got me a SICK ALI @MuhammadAli #ali...,FrankKnuckles3,0.000217,0.014742,0.985041
1,ALI,alinft-official,0.7590,4,20.0,7901.08,1499.229930,2022-05-28 14:11:55+00:00,1530552452646174721,.0029 etc for this (8) 1/1s collection this we...,famousmoonking,0.000259,0.998367,0.001374
2,ALI,alinft-official,0.7590,4,20.0,7901.08,1499.229930,2022-05-26 23:03:16+00:00,1529961393222344704,24 hour pre #MemorialDaySale!!!\n #crypto shri...,famousmoonking,0.000584,0.930651,0.068765
3,ALI,alinft-official,0.7590,4,20.0,7901.08,1499.229930,2022-05-20 14:46:16+00:00,1527661994895020034,hi.\n8 of these @ .01 each for the weekend.\nI...,famousmoonking,0.000171,0.998158,0.001671
4,ALI,alinft-official,0.7590,4,20.0,7901.08,1499.229930,2022-05-10 03:46:15+00:00,1523872016410898432,Limited edition of 9 hand made 1/1 Ali post ca...,famousmoonking,0.000234,0.998247,0.001519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137,CapybarasCountryClub,capybarascountryclub,0.2178,4,6.0,7901.08,430.213806,2022-02-27 21:56:10+00:00,1498054378342735875,My most recent #fanart for #capybarascountrycl...,hxrnyNFT,0.000148,0.997976,0.001876
2138,CapybarasCountryClub,capybarascountryclub,0.2178,4,6.0,7901.08,430.213806,2022-01-07 22:24:59+00:00,1479579847991205894,Do you like what you have seen so far? 👀🔥\n\n#...,CapybarasCC,0.000165,0.001351,0.998484
2139,CapybarasCountryClub,capybarascountryclub,0.2178,4,6.0,7901.08,430.213806,2022-01-07 11:13:52+00:00,1479410953947164674,I love to see our community growing organicall...,CapybarasCC,0.000128,0.001793,0.998079
2140,CapybarasCountryClub,capybarascountryclub,0.2178,4,6.0,7901.08,430.213806,2022-01-06 08:14:47+00:00,1479003498104725507,"GM !!! \n\nNew day, new banner ! \n\nOG roles ...",CapybarasCC,0.000188,0.996583,0.003230
